# EarthCam Data Science Candidate Assignment
### Predicting Weather Data in Joliet, IL in 2024

We are given a historical dataset of weather conditions on which we'll train a ML model to base our temp. predictions.  
Let's begin by importing necessary libraries:

In [1]:
import pandas as pd
import numpy as np

## Data Cleaning and Org.

We want to know:  
- size of the dataset  
- the number of columns / features  
- data types of those columns
- the most relevant features concerning temperature 
- how to represent these features in a way compatible with our machine learning model


In [2]:
weather_data_raw = pd.read_csv("EarthCam_DataScience_Data.csv")
weather_data = weather_data_raw.copy(deep=True)

weather_data.shape

### 27 features

(7324, 27)

In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7324 entries, 0 to 7323
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ObservedAt                     7324 non-null   int64  
 1   ObservedAt_DateTime            7324 non-null   object 
 2   Station                        7324 non-null   object 
 3   Record                         7324 non-null   object 
 4   ID                             7324 non-null   int64  
 5   datetime                       7324 non-null   object 
 6   Temperature                    7324 non-null   object 
 7   Dewpoint                       7324 non-null   object 
 8   FeelsLike                      7324 non-null   object 
 9   HeatIndex                      7324 non-null   object 
 10  Windchill                      7324 non-null   object 
 11  Pressure                       7324 non-null   object 
 12  Precipitation                  7324 non-null   o

#### Notes:  
- We can drop the "daily_total_precipitation" column, as it's completely devoid of data
- Column 16 ("CurrentConditionsAbbreviation") is likely redundant, as we can one-hot encode the "Current Conditions" column anyway
- We're going to have to really clean up the data, as 24 out of 27 columns are objects

Let's take a closer peek at the data itself

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

weather_data.head()

,ObservedAt,ObservedAt_DateTime,Station,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,CurrentConditions,CurrentConditionsAbbreviation,ConditionType,CloudCondition,CloudReport,Visibility,Clouds,Lightning,SunInfo,MoonInfo,raw,daily_total_precipitation
0,1525172100,5/1/2018 5:55,KJOT,011055Z AUTO 19004KT 10SM CLR 13/03 A3000 RMK ...,220160752,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': 37, 'Celsius': 3}","{'Type': None, 'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1015.92, 'InchesOfMercury': 30}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 190, 'Direction': 'So...",0.509,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5463872867481452, 'PhaseName': 'Fu...",AUTO 19004KT 10SM CLR 13/03 A3000 RMK AO2 T013...,NaN
1,1525193700,5/1/2018 11:55,KJOT,011655Z AUTO 23013G23KT 10SM SCT065 27/11 A299...,220215015,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 81, 'Celsius': 27}","{'Fahrenheit': 52, 'Celsius': 11}","{'Type': None, 'Fahrenheit': 81, 'Celsius': 27}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1015.58, 'InchesOfMercury': 29.99}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 230, 'Direction': 'So...",0.368,Scattered,NaN,Cloudy,Scattered,Scattered,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Scattered', 'Coverage': '3/8 -...",[],"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5544029020570569, 'PhaseName': 'Fu...",AUTO 23013G23KT 10SM SCT065 27/11 A2999 RMK AO...,NaN
2,1525215300,5/1/2018 17:55,KJOT,012255Z AUTO 19012G20KT 10SM CLR 28/09 A2991 R...,220262557,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 82, 'Celsius': 28}","{'Fahrenheit': 48, 'Celsius': 9}","{'Type': None, 'Fahrenheit': 82, 'Celsius': 28}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1012.87, 'InchesOfMercury': 29.91}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 190, 'Direction': 'So...",0.303,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5623766330324756, 'PhaseName': 'Fu...",AUTO 19012G20KT 10SM CLR 28/09 A2991 RMK AO2 T...,NaN
3,1525236900,5/1/2018 23:55,KJOT,020455Z AUTO 20013G18KT 10SM CLR 23/11 A2994 R...,220311850,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': 52, 'Celsius': 11}","{'Type': None, 'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1013.89, 'InchesOfMercury': 29.94}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 200, 'Direction': 'So...",0.468,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.5703094498618421, 'PhaseName': 'Wa...",AUTO 20013G18KT 10SM CLR 23/11 A2994 RMK AO2 T...,NaN
4,1525258500,5/2/2018 5:55,KJOT,021055Z AUTO 21008KT 10SM CLR 21/11 A2991 RMK ...,220363232,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 70, 'Celsius': 21}","{'Fahrenheit': 52, 'Celsius': 11}","{'Type': None, 'Fahrenheit': 70, 'Celsius': 21}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1012.87, 'InchesOfMercury': 29.91}","{'Hour': {'In': 0, 'Cm'

In [5]:
weather_data.tail()

,ObservedAt,ObservedAt_DateTime,Station,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,CurrentConditions,CurrentConditionsAbbreviation,ConditionType,CloudCondition,CloudReport,Visibility,Clouds,Lightning,SunInfo,MoonInfo,raw,daily_total_precipitation
7319,1683173700,5/3/2023 23:15,KJOT,040415Z 12003KT 10SM CLR 09/03 A2996 RMK AO2 T...,552954115,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 48, 'Celsius': 9}","{'Fahrenheit': 37, 'Celsius': 3}","{'Type': None, 'Fahrenheit': 48, 'Celsius': 9}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1014.57, 'InchesOfMercury': 29.96}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 120, 'Direction': 'Ea...",0.664,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.44674697934186625, 'PhaseName': 'F...",12003KT 10SM CLR 09/03 A2996 RMK AO2 T00890031,NaN
7320,1683195300,5/4/2023 5:15,KJOT,041015Z 00000KT 10SM CLR 03/02 A2997 RMK AO2 T...,552978176,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 37, 'Celsius': 3}","{'Fahrenheit': 36, 'Celsius': 2}","{'Type': None, 'Fahrenheit': 37, 'Celsius': 3}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1014.9, 'InchesOfMercury': 29.97}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",0.932,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.45516888398704436, 'PhaseName': 'F...",00000KT 10SM CLR 03/02 A2997 RMK AO2 T00280017,NaN
7321,1683214500,5/4/2023 10:35,KJOT,041535Z 23004KT 10SM CLR 17/04 A3002 RMK AO2 T...,552999954,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': 39, 'Celsius': 4}","{'Type': None, 'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1016.6, 'InchesOfMercury': 30.02}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 230, 'Direction': 'So...",0.422,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.46270296909569253, 'PhaseName': 'F...",23004KT 10SM CLR 17/04 A3002 RMK AO2 T01670038,NaN
7322,1683236100,5/4/2023 16:35,KJOT,042135Z 15006G14KT 10SM CLR 23/03 A2994 RMK AO...,553023465,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': 37, 'Celsius': 3}","{'Type': None, 'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1013.89, 'InchesOfMercury': 29.94}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 150, 'Direction': 'So...",0.271,Clear Skies,NaN,Clear,Clear Skies,Clear Skies,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Clear Skies', 'Coverage': '0',...",[],"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.4712322411250427, 'PhaseName': 'Fu...",15006G14KT 10SM CLR 23/03 A2994 RMK AO2 T02270031,NaN
7323,1683260100,5/4/2023 23:15,KJOT,050415Z 16005KT 10SM CLR 15/04 A2997 RMK AO2 T...,553048760,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 59, 'Celsius': 15}","{'Fahrenheit': 39, 'Celsius': 4}","{'Type': None, 'Fahrenheit': 59, 'Celsius': 15}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1014.9, 'InchesOfMercury': 29.97}","{'Hour': {'In': 0,

#### Initial thoughts:  

There are several columns that at first glance warrant being dropped:
- Station: KJOT might be the only unique value  
- Lightning: Are all rows empty brackets??

Further Investigation:
- CurrentConditions, ConditionType, CloudCondition, CloudReport seem to be heavily related. We may be able to drop  
some columns.
- Most of these columns contain multiple features. We'll parse and consolidate these.



In [6]:
### We drop the daily_total_precipitation column as discussed earlier (happens to be the last column)
weather_data = weather_data.iloc[:, :26]

### Check the number of unique values in all columns
columns = list(weather_data.columns)
value_counts = {}
for column_name in columns:
    value_counts[column_name] = len(weather_data[column_name].value_counts())

value_counts

{'ObservedAt': 7217,
 'ObservedAt_DateTime': 7217,
 'Station': 1,
 'Record': 7213,
 'ID': 7217,
 'datetime': 7217,
 'Temperature': 65,
 'Dewpoint': 61,
 'FeelsLike': 165,
 'HeatIndex': 31,
 'Windchill': 76,
 'Pressure': 149,
 'Precipitation': 361,
 'Wind': 1232,
 'RelativeHumidity': 542,
 'CurrentConditions': 27,
 'CurrentConditionsAbbreviation': 13,
 'ConditionType': 3,
 'CloudCondition': 101,
 'CloudReport': 11,
 'Visibility': 11,
 'Clouds': 1332,
 'Lightning': 1,
 'SunInfo': 3613,
 'MoonInfo': 7217,
 'raw': 7212}

We can indeed drop the columns "Station" and "Lightning", as they both have only one unique value throughout all rows

In [7]:
weather_data.drop(["Station","Lightning"], axis=1, inplace=True)
#double checking that we only dropped 3 columns
weather_data.shape

(7324, 24)

### Categorical Columns
The number of unique values in the CurrentConditions, ConditionType, CloudCondition and CloudReport vary.  
Before we drop anything else let's check out these unique values in more detail

In [8]:
### checking the "CurrentConditons" column
weather_data["CurrentConditions"].value_counts()

CurrentConditions
Clear Skies                             3335
Overcast Sky                            1176
Scattered                               1112
Broken Sky                               615
Mist                                     398
Light Rain                               181
Rain                                     122
Light Snow                               119
Light Drizzle                             60
Drizzle                                   49
Fog                                       42
Strong Rain                               33
Haze                                      15
In The Vicinity, Thunderstorms            13
/ Overcast Sky                            10
Freezing, Fog                              7
Snow                                       7
Light Thunderstorms, Rain                  5
Thunderstorms                              5
Strong Thunderstorms, Rain                 4
Thunderstorms, Rain                        4
Dry                                  

Notes:

- High cardinality of values, qualitative descriptions of precipitation unnecessary?

In [9]:
### checking the "ConditionType" column
weather_data["ConditionType"].value_counts()

ConditionType
Cloudy    3798
Clear     3525
Rain         1
Name: count, dtype: int64

In [10]:
### checking the "CloudCondition" column
weather_data["CloudCondition"].value_counts()

CloudCondition
Clear Skies                                                               3335
Overcast Sky                                                              1176
Scattered                                                                  678
Broken Sky                                                                 323
Broken Sky / Overcast Sky                                                  289
Scattered / Overcast Sky                                                   193
Mist / Overcast Sky                                                        159
Scattered / Broken Sky                                                     144
Mist / Clear Skies                                                         128
Scattered / Broken Sky / Overcast Sky                                       97
Light Rain / Overcast Sky                                                   69
Light Snow / Overcast Sky                                                   61
Rain / Overcast Sky                  

This column has very high cardinality, is telling too many different "stories"

In [11]:
### checking the "CloudReport" column
weather_data["CloudReport"].value_counts()

CloudReport
Clear Skies                              3475
Overcast Sky                             1636
Scattered                                 711
Broken Sky / Overcast Sky                 433
Broken Sky                                355
Scattered / Overcast Sky                  286
Scattered / Broken Sky / Overcast Sky     197
Scattered / Broken Sky                    177
Vertical Visibility                        51
Broken Sky / Scattered / Overcast Sky       2
Broken Sky / Scattered                      1
Name: count, dtype: int64


We want to avoid highly cardinal categorical columns as they bloat the feature space and encourage overfitting.  
The "CloudCondition" and "CurrentCondition"s" columns have extensive qualitative descriptions of precipitation  
that would probably be redundant seeing as we have a dedicated "precipitation" column already.  

Being that the standard descriptors of cloud cover are Clear, Scattered, Broken, and Overcast,  
we only realistically need these four values to represent weather conditions.

As such, I have elected to drop "CloudReport", "ConditionType", and "CloudCondition" columns,   
as well as the "CurrentConditionAbbreviation" discussed earlier  
keeping only the "CurrentConditions" column, as this column has the neatest separation between cloud cover values.     
In addition, I want to replace all other values in this column with "Precipitation," as all other values in the column   
describe inclement weather. In this way, we reduce the dimensionality of this "weather conditions" feature space while   
preserving the aggregate spread of the weather conditions




In [12]:
### Drop the "CloudReport", "ConditionType", "CloudCondition", and "CurrentConditionsAbbreviation" columns
weather_data.drop(["CloudReport", "ConditionType", "CloudCondition", "CurrentConditionsAbbreviation"], axis=1, inplace=True)

As the column info for the "CurrentConditions" showed, some of the values have a "/" prefix in front of them.

In [13]:
### Example
weather_data["CurrentConditions"][1404]

'/ Overcast Sky'

In [14]:
### Consolidating cloud cover values with a forward slash prefix
row_vals_to_rename = ['/ Overcast Sky', '/ Broken Sky', '/ Scattered']
renaming_dict = {}

for i in range(len(weather_data)):
    current_cond = weather_data["CurrentConditions"].iloc[i]
    if current_cond in row_vals_to_rename:
        renaming_dict[i] = current_cond[2:]

for index in renaming_dict.keys():
    weather_data["CurrentConditions"].iloc[index] = renaming_dict[index]

/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/2607673460.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["CurrentConditions"].iloc[index] = renaming_dict[index]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/2607673460.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["CurrentConditions"].iloc[index] = renaming_dict[index]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/2607673460.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [15]:
weather_data.iloc[1404]

ObservedAt                                                    1555260900
ObservedAt_DateTime                                      4/14/2019 11:55
Record                 141655Z AUTO 36015G22KT 1 1/2SM UP OVC007 01/0...
ID                                                             292169946
datetime               {'timezone': 'America/Chicago', 'string_local'...
Temperature                             {'Fahrenheit': 34, 'Celsius': 1}
Dewpoint                                {'Fahrenheit': 34, 'Celsius': 1}
FeelsLike              {'Type': 'Windchill', 'Fahrenheit': 23, 'Celsi...
HeatIndex                          {'Fahrenheit': None, 'Celsius': None}
Windchill                              {'Fahrenheit': 23, 'Celsius': -5}
Pressure                 {'Millibar': 1003.73, 'InchesOfMercury': 29.64}
Precipitation          {'Hour': {'In': 0.19, 'Cm': 0.48}, 'Snow': {'I...
Wind                   {'Direction': {'Degree': 360, 'Direction': 'No...
RelativeHumidity                                   

Despite the SettingWithCopy Warning we recieved, it doesn't seem to have affected the operations so far. 
The value of the edited columns look good.   
We proceed cautiously

In [16]:
### Change all other row values to "Precipitation"
cloud_cover_vals = list(weather_data["CurrentConditions"].value_counts().index)[:4] # the top 4 values by occurrence happen to be the 4 cloud cover descriptors
weather_data.loc[~weather_data["CurrentConditions"].isin(cloud_cover_vals), "CurrentConditions"] = "Precipitation"

### confirming our changes
weather_data["CurrentConditions"].value_counts()


CurrentConditions
Clear Skies      3335
Overcast Sky     1186
Scattered        1113
Precipitation    1072
Broken Sky        618
Name: count, dtype: int64

Let's now check out the state of the dataset currently

In [17]:
weather_data.sample(3)

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,CurrentConditions,Visibility,Clouds,SunInfo,MoonInfo,raw
4041,1612785300,2/8/2021 5:55,081155Z AUTO 00000KT 5SM -SN SCT030 BKN037 OVC...,432194521,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 5, 'Celsius': -15}","{'Fahrenheit': -2, 'Celsius': -19}","{'Type': None, 'Fahrenheit': 5, 'Celsius': -15}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1027.43, 'InchesOfMercury': 30.34}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",0.718,Precipitation,"[{'Miles': 5, 'Kilometers': 8, 'Direction': No...","[{'Condition': 'Scattered', 'Coverage': '3/8 -...","{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.8854540134491012, 'PhaseName': 'Wa...",AUTO 00000KT 5SM -SN SCT030 BKN037 OVC055 M15/...
176,1528973700,6/14/2018 5:55,141055Z AUTO 00000KT 3SM BR CLR 17/16 A3007 RM...,229046531,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': 61, 'Celsius': 16}","{'Type': None, 'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1018.29, 'InchesOfMercury': 30.07}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",0.939,Precipitation,"[{'Miles': 3, 'Kilometers': 4.8, 'Direction': ...","[{'Condition': 'Clear Skies', 'Coverage': '0',...","{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.024438870470191453, 'PhaseName': '...",AUTO 00000KT 3SM BR CLR 17/16 A3007 RMK AO2 T0...
2883,1588136100,4/28/2020 23:55,290455Z AUTO 20004KT 10SM RA SCT027 BKN065 OVC...,372703807,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': 59, 'Celsius': 15}","{'Type': None, 'Fahrenheit': 63, 'Celsius': 17}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1001.7, 'InchesOfMercury': 29.58}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 200, 'Direction': 'So...",0.881,Precipitation,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...","[{'Condition': 'Scattered', 'Coverage': '3/8 -...","{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.1938348583801257, 'PhaseName': 'Fi...",AUTO 20004KT 10SM RA SCT027 BKN065 OVC085 17/1...


### Quantitative Columns

Intial Notes:
- I will extract Farenheight values with the idea that if Celsius is requested, it's much simpler to convert from F to C rather than  
store each Celsius temperature individually
- The values within are strings, but can defintely be evaluated as dictionaries.
- The "Feels Like," "Heat Index," and "Windchill" columns look like they could be consolidated somehow.  
"Feels Like" should be a numerical column, while we can encode "Heat Index" and "Windchill" columns into yes and no (1 and 0)  
This will work as the "Feels Like" temp is reflected in the "Heat Index" or "Windchill" column (whichever one, if any, are "active" (1))
##### before we move on however, upon further examination
- We might have not needed to do all that work on the categorical columns, as there exists a standalone cloud column.  
containing the cloud cover values. If the "Condition" values within the cloud column match that of the   
"CurrentConditions" column, we might be able to drop the "CurrentConditions" column.  An added benefit of this is that we will   
separate out cloud cover and precipitation from each other (in the "CurrentConditions" column precipitation is  
mutually exclusive from the level of cloud cover)

Let's check on the relationship between the conditions in the "Clouds" column and the values of the "CurrentConditions" column

In [18]:
### double check the data contained within "Clouds"
clouds = weather_data["Clouds"]

In [19]:
type(clouds[42])

str

In [20]:
### since each item is a string, we can use eval() to convert to dictionaries and lists and extract the "Condition" for each row

### initialize empty list
# cloud_conditions = []

# for i in range(len(clouds)):
#     print(i)
#     values = eval(clouds[i])[0] #cloud dictionary is contained within list brackets, hence the index
#     cloud_conditions.append(values["Condition"])


We got an error in item at index 2266, let's check it out!

In [21]:
clouds[2266]

'[]'

vs

In [22]:
clouds[2267]

"[{'Condition': 'Clear Skies', 'Coverage': '0', 'Abbreviation': 'CLR', 'Altitude': {'Feet': None, 'Meters': None}}]"

It seems that row 2266 is missing all values missing values in this column.

In [23]:
# missing_incomplete = {"Clouds" : {"complete" : 0, "incomplete" : 0, "missing" : 0}}

### reinitialize the cloud conditions list
cloud_conditions = []

for i in range(len(clouds)):
    values = clouds[i]
    if values[0] != "[":
        # missing_incomplete["Clouds"]["incomplete"] += 1
        cloud_conditions.append(values)
    else:
        values = eval(values)
        try:
            cloud_conditions.append(values[0]["Condition"]) #cloud dictionary is contained within list brackets, hence the 0 index
            # missing_incomplete["Clouds"]["complete"] += 1
        except IndexError:
            # missing_incomplete["Clouds"]["missing"] += 1
            cloud_conditions.append("NaN")

Ok. before we go counting the missing values for every column, let's first stay on track   
and compare the "CurrentConditions" column values and the "Condition" value in the  
 "Clouds" column see how similar they are, and whether we can drop one of them.

In [24]:
cloud_conditions_df = pd.DataFrame(cloud_conditions, columns=["CloudConditions"])
current_conditions_df = pd.DataFrame(weather_data_raw["CurrentConditions"]) #we've got to use the untouched data set since we 
                                                                            #changed values in CurrentConditions already


cloud_conditions_df.value_counts()


CloudConditions    
Clear Skies            3472
Overcast Sky           1636
Scattered              1371
Broken Sky              791
Vertical Visibility      51
NaN                       3
Name: count, dtype: int64

In [25]:
current_conditions_df.value_counts()

CurrentConditions                   
Clear Skies                             3335
Overcast Sky                            1176
Scattered                               1112
Broken Sky                               615
Mist                                     398
Light Rain                               181
Rain                                     122
Light Snow                               119
Light Drizzle                             60
Drizzle                                   49
Fog                                       42
Strong Rain                               33
Haze                                      15
In The Vicinity, Thunderstorms            13
/ Overcast Sky                            10
Freezing, Fog                              7
Snow                                       7
Light Thunderstorms, Rain                  5
Thunderstorms                              5
Strong Thunderstorms, Rain                 4
Thunderstorms, Rain                        4
Dry               

I like the look of the CloudConditions data better as it focuses on cloud conditions and doesn't overlap with precipitation, unlike the "CurrentConditions" column. We will depend on the precipitation column to track precipitation.

So now our immediate plan of action is:
- use the currentConditions column to fill in the NaN values of the cloud conditions column
- drop the currentConditions columns

In [26]:
# getting the rows with NaN vals
cloud_condition_NaN_vals = cloud_conditions_df[cloud_conditions_df["CloudConditions"] == "NaN"]

### let's search for these NaN val indices in the CurrentConditions df
current_conditions_df.iloc[list(cloud_condition_NaN_vals.index)]


,CurrentConditions
2266,Dry
2338,Dry
5474,Dry


Based on the above, I am going to replace the NaN values in our CloudConditions column with the value "Clear Skies." This should be fine  
as "Clear Skies" is the value mode by a large margin, and corresponds cleanly with the "Dry" value we see at the same indices in "CurrentConditions"

In [27]:
cloud_conditions_df[cloud_conditions_df["CloudConditions"] == "NaN"] = "Clear Skies"
cloud_conditions_df.value_counts()

CloudConditions    
Clear Skies            3475
Overcast Sky           1636
Scattered              1371
Broken Sky              791
Vertical Visibility      51
Name: count, dtype: int64

The value "Vertical Visibility" is bothering me. At face value it seems analagous to "Clear Skies". Let's see what's going on in the  
"Current Conditions" column at the same indices.

In [28]:
cloud_condition_VV_vals = cloud_conditions_df[cloud_conditions_df["CloudConditions"] == "Vertical Visibility"]

### searching for these val's indices within "CurrentConditions" column
current_conditions_df.iloc[list(cloud_condition_VV_vals.index)]

,CurrentConditions
88,Fog
232,Mist
356,Fog
372,Mist
384,Mist
388,Fog
408,Mist
419,Mist
424,Mist
448,Mist


As evidenced above, "VerticalVisibility" indicates low visibility conditions. Let's rename "Vertical Visibility" to "Low Visibility." Then we can proceed with dropping the "CurrentConditions" column. We will rename the "Clouds" column to Conditions at this point, using our temporary cloud_conditions_df  
to overwrite the existing data.  
  
As shown below, we will be replacing the "CloudConditions" values with only the "Condition" value, as "Coverage" is 100% correlated to Condition, "Abbreviation" offers no additional richness, and "Altitude" presumably refers to the cloud base height, which is a function of dew point and temperature, both of which are included in the dataset; we can assume (or hope) that this latent relationship is represented in the data. Not too sure if this is the right move, however.

In [29]:
### renaming the "Vertical Visibility" values to "Low Visibility"
cloud_conditions_df[cloud_conditions_df["CloudConditions"] == "Vertical Visibility"] = "Low Visibility"

### double check "Clouds" and cloud_conditions_df's lengths line up

print(len(weather_data["Clouds"]) == len(cloud_conditions_df))

### replacing the "Clouds" column with our custom "Conditions" column (cloud_conditions_df)

weather_data.rename(columns={"Clouds" : "Conditions"}, inplace=True)

weather_data["Conditions"] = cloud_conditions_df

### dropping the "CurrentConditions" column
weather_data.drop(["CurrentConditions"], axis=1, inplace=True)

weather_data.sample(5)

True


,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility,Conditions,SunInfo,MoonInfo,raw
292,1531392900,7/12/2018 5:55,121055Z AUTO 00000KT 10SM CLR 19/18 A3014 RMK ...,234661550,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 66, 'Celsius': 19}","{'Fahrenheit': 64, 'Celsius': 18}","{'Type': None, 'Fahrenheit': 66, 'Celsius': 19}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1020.66, 'InchesOfMercury': 30.14}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",0.940,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...",Clear Skies,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.9734689772395934, 'PhaseName': 'Ne...",AUTO 00000KT 10SM CLR 19/18 A3014 RMK AO2 T018...
5778,1649954100,4/14/2022 11:35,141635Z 25017G29KT 10SM CLR 08/M03 A2990 RMK A...,515437564,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 46, 'Celsius': 8}","{'Fahrenheit': 27, 'Celsius': -3}","{'Type': 'Windchill', 'Fahrenheit': 38, 'Celsi...","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': 38, 'Celsius': 3}","{'Millibar': 1012.53, 'InchesOfMercury': 29.9}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 250, 'Direction': 'We...",0.461,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...",Clear Skies,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.42411747916272485, 'PhaseName': 'W...",25017G29KT 10SM CLR 08/M03 A2990 RMK AO2 T0080...
2497,1579262100,1/17/2020 5:55,171155Z AUTO 00000KT 10SM CLR M08/M12 A3075 RM...,350834831,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 18, 'Celsius': -8}","{'Fahrenheit': 10, 'Celsius': -12}","{'Type': None, 'Fahrenheit': 18, 'Celsius': -8}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1041.32, 'InchesOfMercury': 30.75}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",0.732,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...",Clear Skies,"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.7478179220634058, 'PhaseName': 'Th...",AUTO 00000KT 10SM CLR M08/M12 A3075 RMK AO2 7/...
3816,1608098100,12/15/2020 23:55,160555Z AUTO 08005KT 10SM CLR M02/M07 A3015 RM...,420745960,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 28, 'Celsius': -2}","{'Fahrenheit': 19, 'Celsius': -7}","{'Type': 'Windchill', 'Fahrenheit': 21, 'Celsi...","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': 21, 'Celsius': -6}","{'Millibar': 1021, 'InchesOfMercury': 30.15}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 80, 'Direction': 'Eas...",0.690,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...",Clear Skies,"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.058682980949757435, 'PhaseName': '...",AUTO 08005KT 10SM CLR M02/M07 A3015 RMK AO2 T1...
6219,1659303300,7/31/2022 16:35,312135Z 20010KT 10SM SCT050 28/18 A3001 RMK AO...,525961985,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 82, 'Celsius': 28}","{'Fahrenheit': 64, 'Celsius': 18}","{'Type': 'HeatIndex', 'Fahrenheit': 84, 'Celsi...","{'Fahrenheit': 84, 'Celsius': 29}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1016.26, 'InchesOfMercury': 30.01}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 200, 'Direction': 'So...",0.546,"[{'Miles': 10, 'Kilometers': 16.1, 'Direction'...",Scattered,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.09841153682501284, 'PhaseName': 'W...",20010KT 10SM SCT050 28/18 A3001 RMK AO2 T02750176


OK. Now we have a rough framework on how to handle most of the data.  
Let's now start examining each columns' data to see which items we are  
interested in using for our model.

#### Column: Visibility

In [30]:
weather_data.iloc[1209]["Visibility"]

"[{'Miles': 10, 'Kilometers': 16.1, 'Direction': None, 'DirectionShort': None}]"

Thoughts:
- We'll stick with 'Miles' as our distance unit.
- Let's see how many unique values there are for 'Direction.' DirectionShort is probably superfluous.

In [31]:
# we'll parse through each entry and extract the directions, then we'll turn the list into a dataframe to count unique values
directions = []

for i in range(len(weather_data)):
    visibility_val = eval(weather_data.iloc[i]["Visibility"])[0]
    direction = visibility_val["Direction"]
    directions.append(direction)


In [32]:
directions

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

It seems like the "Direction" value had no data. Let's leave it and the "DirectionShort" values out. As discussed earlier, we are keeping the "Miles" value

In [33]:
weather_data["Visibility"][0]

"[{'Miles': 10, 'Kilometers': 16.1, 'Direction': None, 'DirectionShort': None}]"

In [34]:
### extracting miles from the "Visibility" column

### initialize blank Miles list

miles = []

for i in range(len(weather_data)):
    vis_data = eval(weather_data["Visibility"].iloc[i])[0]
    miles.append(float(vis_data["Miles"]))

### confirmng miles Miles and weather data match up lengthwise

print(len(miles) == len(weather_data))

weather_data["Visibility"] = miles

### Change the column name to specify Visibility in Miles

weather_data.rename(columns={"Visibility" : "Visibility(mi)"}, inplace=True)

weather_data.sample(3)

True


,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility(mi),Conditions,SunInfo,MoonInfo,raw
3754,1606758900,11/30/2020 11:55,301755Z AUTO 32011G17KT 10SM OVC044 01/M07 A29...,417501155,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 34, 'Celsius': 1}","{'Fahrenheit': 19, 'Celsius': -7}","{'Type': 'Windchill', 'Fahrenheit': 25, 'Celsi...","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': 25, 'Celsius': -4}","{'Millibar': 1015.58, 'InchesOfMercury': 29.99}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 320, 'Direction': 'No...",0.555,10.0,Overcast Sky,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5104836450311355, 'PhaseName': 'Fu...",AUTO 32011G17KT 10SM OVC044 01/M07 A2999 RMK A...
3943,1610754900,1/15/2021 17:55,152355Z AUTO 00000KT 1/2SM FZFG VV005 M02/M02 ...,427235690,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 28, 'Celsius': -2}","{'Fahrenheit': 28, 'Celsius': -2}","{'Type': None, 'Fahrenheit': 28, 'Celsius': -2}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1001.36, 'InchesOfMercury': 29.57}","{'Hour': {'In': 0.01, 'Cm': 0.03}, 'Snow': {'I...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",1.000,1.0,Low Visibility,"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.09716764838240566, 'PhaseName': 'W...",AUTO 00000KT 1/2SM FZFG VV005 M02/M02 A2957 RM...
1711,1561805700,6/29/2019 5:55,291055Z AUTO 00000KT 7SM CLR 20/20 A3011 RMK A...,308093077,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 68, 'Celsius': 20}","{'Fahrenheit': 68, 'Celsius': 20}","{'Type': None, 'Fahrenheit': 68, 'Celsius': 20}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1019.65, 'InchesOfMercury': 30.11}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 0, 'Direction': 'Nort...",1.000,7.0,Clear Skies,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.8796035560187412, 'PhaseName': 'Wa...",AUTO 00000KT 7SM CLR 20/20 A3011 RMK AO2 T0203...


Visibility is done. Let's now move on to

#### Column: SunInfo

In [35]:
weather_data.head(4)

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility(mi),Conditions,SunInfo,MoonInfo,raw
0,1525172100,5/1/2018 5:55,011055Z AUTO 19004KT 10SM CLR 13/03 A3000 RMK ...,220160752,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': 37, 'Celsius': 3}","{'Type': None, 'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1015.92, 'InchesOfMercury': 30}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 190, 'Direction': 'So...",0.509,10.0,Clear Skies,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5463872867481452, 'PhaseName': 'Fu...",AUTO 19004KT 10SM CLR 13/03 A3000 RMK AO2 T013...
1,1525193700,5/1/2018 11:55,011655Z AUTO 23013G23KT 10SM SCT065 27/11 A299...,220215015,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 81, 'Celsius': 27}","{'Fahrenheit': 52, 'Celsius': 11}","{'Type': None, 'Fahrenheit': 81, 'Celsius': 27}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1015.58, 'InchesOfMercury': 29.99}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 230, 'Direction': 'So...",0.368,10.0,Scattered,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5544029020570569, 'PhaseName': 'Fu...",AUTO 23013G23KT 10SM SCT065 27/11 A2999 RMK AO...
2,1525215300,5/1/2018 17:55,012255Z AUTO 19012G20KT 10SM CLR 28/09 A2991 R...,220262557,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 82, 'Celsius': 28}","{'Fahrenheit': 48, 'Celsius': 9}","{'Type': None, 'Fahrenheit': 82, 'Celsius': 28}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1012.87, 'InchesOfMercury': 29.91}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 190, 'Direction': 'So...",0.303,10.0,Clear Skies,"{'IsSunUp': True, 'Sunrise': {'timezone': 'Ame...","{'Phase': 0.5623766330324756, 'PhaseName': 'Fu...",AUTO 19012G20KT 10SM CLR 28/09 A2991 RMK AO2 T...
3,1525236900,5/1/2018 23:55,020455Z AUTO 20013G18KT 10SM CLR 23/11 A2994 R...,220311850,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': 52, 'Celsius': 11}","{'Type': None, 'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1013.89, 'InchesOfMercury': 29.94}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 200, 'Direction': 'So...",0.468,10.0,Clear Skies,"{'IsSunUp': False, 'Sunrise': {'timezone': 'Am...","{'Phase': 0.5703094498618421, 'PhaseName': 'Wa...",AUTO 20013G18KT 10SM CLR 23/11 A2994 RMK AO2 T...


In [36]:
weather_data["SunInfo"][0]

"{'IsSunUp': True, 'Sunrise': {'timezone': 'America/Chicago', 'string_local': '20180501054911', 'string_utc': '20180501104911', 'iso_8601_local': '2018-05-01T05:49:11-05:00', 'unix_timestamp': 1525171751}, 'Sunset': {'timezone': 'America/Chicago', 'string_local': '20180501195025', 'string_utc': '20180502005025', 'iso_8601_local': '2018-05-01T19:50:25-05:00', 'unix_timestamp': 1525222225}}"

I think the most important features present here are the Sunrise and Sunset times, as earlier sunrises could correlate with higher temperatures and vice versa, etc.  
##### Objective:
- split SunInfo into SunRise and SunSet. We will use the iso_8601 fields to grab the 24hr clock (will probably use only the hour for simplicity's sake)  

In [37]:
### initialize empty sunrise and sunset arrays

sunrise_times, sunset_times = [], []

# iterate through, grabbing sunrise and sunset times

for i in range(len(weather_data)):
    sun_info = eval(weather_data["SunInfo"].iloc[i])

    sunrise_time = sun_info['Sunrise']['iso_8601_local'][11:13]
    sunrise_times.append(sunrise_time)
    
    sunset_time = sun_info['Sunset']['iso_8601_local'][11:13]
    sunset_times.append(sunset_time)

sunrise_times_df = pd.DataFrame(sunrise_times)
sunset_times_df = pd.DataFrame(sunset_times)

print("Sunrise Times\n")
print(sunrise_times_df.value_counts())

print("Sunset Times\n")
print(sunset_times_df.value_counts())

print("\nAll Lengths Match\n",len(sunrise_times_df) == len(sunset_times_df) == len(weather_data))

### we are going to straight up drop the "SunInfo" column, after inseritng two new columns from our sun dataframes after it

sun_info_index = list(weather_data.columns).index('SunInfo')

weather_data.insert(sun_info_index, "SunSet(HH)", sunset_times)
weather_data.insert(sun_info_index, "SunRise(HH)", sunrise_times)
weather_data.drop(["SunInfo"], axis=1, inplace=True)

weather_data.sample()

Sunrise Times

06    3085
05    2293
07    1946
Name: count, dtype: int64
Sunset Times

19    1889
20    1867
16    1641
17    1091
18     836
Name: count, dtype: int64

All Lengths Match
 True


,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH),MoonInfo,raw
5048,1634360100,10/15/2021 23:55,160455Z AUTO 29005KT 10SM CLR 12/07 A2994 RMK ...,484543853,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 54, 'Celsius': 12}","{'Fahrenheit': 45, 'Celsius': 7}","{'Type': None, 'Fahrenheit': 54, 'Celsius': 12}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1013.89, 'InchesOfMercury': 29.94}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 290, 'Direction': 'We...",0.717,10.0,Clear Skies,07,18,"{'Phase': 0.35635485716955817, 'PhaseName': 'W...",AUTO 29005KT 10SM CLR 12/07 A2994 RMK AO2 T011...


# Note for the near future 
I'm interested to see if my hypothesis about earlier sunrise times correlating with higher temperatures holds any weight. We'll see once I get all this data cleaned

Ok. Now we move on to  
  
  #### Column: MoonInfo

In [38]:
weather_data['MoonInfo'].iloc[0]

"{'Phase': 0.5463872867481452, 'PhaseName': 'Full Moon', 'Illumination': 0.9789126937625077, 'Age': 16.14, 'IsMoonUp': True, 'Moonrise': {'timezone': 'America/Chicago', 'string_local': '19691231180000', 'string_utc': '19700101000000', 'iso_8601_local': '1969-12-31T18:00:00-06:00', 'unix_timestamp': False}, 'Moonset': {'timezone': 'America/Chicago', 'string_local': '20230607084000', 'string_utc': '20230607134000', 'iso_8601_local': '2023-06-07T08:40:00-05:00', 'unix_timestamp': 1686145200}}"

I think it's safe to drop the "MoonInfo" column, as I don't see how the phase of the moon would affect temperature on Earth (beyond fractionally, anyway)  
Maybe I'm wrong, but I believe there are more relevant features in this dataset than this.

In [39]:
weather_data.drop(['MoonInfo'], axis=1, inplace=True)

Let's drop the 'raw' column too, while we're at it

In [40]:
weather_data.drop(['raw'], axis=1, inplace=True)

In [41]:
weather_data.sample()

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
5555,1645054500,2/16/2022 17:35,162335Z 20021G28KT 10SM BKN043 OVC055 13/09 A2...,509739354,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': 48, 'Celsius': 9}","{'Type': None, 'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1009.15, 'InchesOfMercury': 29.8}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...","{'Direction': {'Degree': 200, 'Direction': 'So...",0.769,10.0,Broken Sky,06,17


Moving back from the middle, next up is  
  
#### Column: Wind

In [42]:
weather_data["Wind"].iloc[256]

"{'Direction': {'Degree': 0, 'Direction': 'North', 'DirectionShort': 'N', 'VariableDegreeBegin': None, 'VariableDegreeEnd': None}, 'Speed': {'Knots': 0, 'MilesPerHour': 0, 'KilometersPerHour': 0, 'Condition': 'Calm'}, 'Gust': {'Knots': None, 'MilesPerHour': None, 'KilometersPerHour': None}}"

Here I will take the "Direction" and "Speed" (in MilesPerHour), my thinking being that given Joliet's proximity to the Great Lakes and Canada,  
winds coming from the North may influence temperatures to go down, and vice versa with wind blowing from the South.  
We will probably one hot encode these values, as we don't want an arbitrary ordering of ordinal directions influence the model calculations in any way  

### Note for the near future

one hot encode the direction column

In [43]:
### initializing empty lists for direction and speed

directions, speeds = [], []

# same as before, iterate through and grab values as we go

for i in range(len(weather_data)):
    wind_data = eval(weather_data["Wind"].iloc[i])
    directions.append(wind_data["Direction"]['Direction'])
    speeds.append(wind_data['Speed']['MilesPerHour'])

directions_df = pd.DataFrame(directions)
speeds_df = pd.DataFrame(speeds)

print("Directions\n")
print(directions_df.value_counts())

Directions

North             2162
West               647
SouthSouthWest     580
South              518
WestSouthWest      394
NorthEast          340
WestNorthWest      328
SouthWest          308
East               272
SouthSouthEast     271
NorthNorthEast     263
NorthWest          254
NorthNorthWest     252
SouthEast          239
EastNorthEast      235
EastSouthEast      220
Name: count, dtype: int64


We are going to reduce all directions to the four cardinal directions (NSEW) to decrease the dimensianality of the feature space.  
Each compound direction will be reduced to the first direction that appears

In [44]:
cardinal_directions = ["North","East","West","South"]
compound_directions_df = directions_df[~directions_df[0].isin(cardinal_directions)]#parses values not in cardinal_directions
compound_directions_df.isnull().value_counts()

False    3684
True       41
Name: count, dtype: int64

We have 41 null values in our compound directions df. Before we make any decisions, let's see these rows in context.  
I will gather the indices of these null value rows to join with the main dataset

In [45]:
# consolidating indices of null wind direction values
null_directions_indices = compound_directions_df[compound_directions_df[0].isnull()]
null_directions_indices = list(null_directions_indices.index)

In [46]:
# weather_data.iloc[null_directions_indices]

Let's look roughly a year back and identify what, if any, the most common wind directions were

In [47]:
minus365_null_direction_indices = [index-1480 for index in null_directions_indices]#many days have multiple measurements of temp, so we have to go back more than 365 entries
minus365_weather_data = weather_data.iloc[minus365_null_direction_indices]

### initializing empty list for last year's directions

min365_directions = []

# same as before, iterate through and grab values as we go

for i in list(minus365_weather_data.index):

    wind_data = eval(weather_data['Wind'].iloc[i])
    min365_directions.append(wind_data["Direction"]['Direction'])


### display the wind direction a year before each row instance of None  
pd.DataFrame(min365_directions).value_counts()

North             15
SouthSouthWest     6
West               5
WestSouthWest      4
NorthNorthEast     2
NorthNorthWest     2
NorthWest          2
SouthSouthEast     2
NorthEast          1
SouthWest          1
WestNorthWest      1
Name: count, dtype: int64

I am torn between replacing the whole slew of None values with North, as it's clearly the most frequent, or to copy the Direction values from last year, which seems  
problematic but also plausible, as the date range is the same.

I decided to replace all null values in compound_direction_df with "North", for simplicity's sake.

In [48]:
compound_directions_df[compound_directions_df.isnull()] = "North"

/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/872082825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compound_directions_df[compound_directions_df.isnull()] = "North"
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/872082825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compound_directions_df[compound_directions_df.isnull()] = "North"


In [49]:
compound_directions_df.value_counts()

SouthSouthWest    580
WestSouthWest     394
NorthEast         340
WestNorthWest     328
SouthWest         308
SouthSouthEast    271
NorthNorthEast    263
NorthWest         254
NorthNorthWest    252
SouthEast         239
EastNorthEast     235
EastSouthEast     220
North              41
Name: count, dtype: int64

Now we break down each compound direction into it's dominant(first) direction using a fancy regex (joking)

In [50]:
for i in range(len(compound_directions_df)):
    # print(i)
    direction = compound_directions_df.iloc[i].values[0]
    # print(direction)
    if direction[:5] == "North":
        compound_directions_df.iloc[i] = "North"
    elif direction[:5] == "South":
        compound_directions_df.iloc[i] = "South"
    elif direction[:4] == "East":
        compound_directions_df.iloc[i] = "East"
    elif direction[:4] == "West":
        compound_directions_df.iloc[i] = "West"

/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/3887444348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compound_directions_df.iloc[i] = "South"
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/3887444348.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compound_directions_df.iloc[i] = "West"
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/3887444348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [51]:
compound_directions_df.value_counts()

South    1398
North    1150
West      722
East      455
Name: count, dtype: int64

In [52]:
compound_dir_indices = {}
for i in range(len(compound_directions_df)):
    compound_dir_indices[i] = list(compound_directions_df.index)[i]


In [53]:
compound_directions_df[0] ### name of column not specified, use 0

1       South
3       South
4       South
5        West
9       South
        ...  
7317    North
7319     East
7321    South
7322    South
7323    South
Name: 0, Length: 3725, dtype: object

In [54]:
# reintegrate adjusted values to the main dataset
for key in compound_dir_indices.keys():
    weather_index = compound_dir_indices[key]
    weather_data['Wind'].iloc[weather_index] = compound_directions_df[0].iloc[key]

/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/1737884161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['Wind'].iloc[weather_index] = compound_directions_df[0].iloc[key]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/1737884161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['Wind'].iloc[weather_index] = compound_directions_df[0].iloc[key]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/1737884161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

We've replaced the compound directions with the reduced directions in the "Wind column."
Before we rename the column to WindDirection, we must add the single directions to this column.  
Affer that, we will add in WindSpeeds as a column and drop the original Wind column.  
This is defintely a messy way to do it, but we need to keep moving.

In [55]:
single_directions_df = directions_df[directions_df[0].isin(cardinal_directions)]

single_dir_indices = {}
for i in range(len(single_directions_df)):
    single_dir_indices[i] = list(single_directions_df.index)[i]

# reintegrate adjusted values to the main dataset
for key in single_dir_indices.keys():
    weather_index = single_dir_indices[key]
    weather_data['Wind'].iloc[weather_index] = single_directions_df[0].iloc[key]

/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/864760919.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['Wind'].iloc[weather_index] = single_directions_df[0].iloc[key]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/864760919.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['Wind'].iloc[weather_index] = single_directions_df[0].iloc[key]
/var/folders/r3/1tm0xhts0ynf16jd99d8nwwm0000gn/T/ipykernel_59614/864760919.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [56]:
weather_data.sample()

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,Wind,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
7038,1676913300,2/20/2023 11:15,201715Z 14005KT 10SM CLR 07/M01 A2978 RMK AO2 ...,546162981,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 45, 'Celsius': 7}","{'Fahrenheit': 30, 'Celsius': -1}","{'Type': 'Windchill', 'Fahrenheit': 42, 'Celsi...","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': 42, 'Celsius': 6}","{'Millibar': 1008.47, 'InchesOfMercury': 29.78}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...",South,0.571,10.0,Clear Skies,06,17


In [57]:
weather_data.rename(columns={"Wind" : "WindDirection"}, inplace=True)
weather_data.insert(12, "WindSpeed(mph)", speeds_df)
weather_data.sample()

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,Precipitation,WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
399,1533617700,8/6/2018 23:55,070455Z AUTO 00000KT 7SM OVC110 23/23 A3002 RM...,239878813,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': 73, 'Celsius': 23}","{'Type': None, 'Fahrenheit': 73, 'Celsius': 23}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1016.6, 'InchesOfMercury': 30.02}","{'Hour': {'In': 0, 'Cm': 0}, 'Snow': {'In': 0,...",0.0,North,1.0,7.0,Overcast Sky,05,20


Phew. That was a doozy. Let's move on to

#### Column: Precipitation

In [58]:
eval(list(weather_data.sample()["Precipitation"])[0])

{'Hour': {'In': 0, 'Cm': 0},
 'Snow': {'In': 0, 'Cm': 0},
 'Year': {'In': 6.06, 'Cm': 15.39},
 'Month': {'In': 0.42, 'Cm': 1.07},
 'Week': {'In': 0, 'Cm': 0},
 'Day': {'In': 0, 'Cm': 0},
 'Year_snow': {'In': 0, 'Cm': 0},
 'Month_snow': {'In': 0, 'Cm': 0},
 'Week_snow': {'In': 0, 'Cm': 0},
 'Day_snow': {'In': 0, 'Cm': 0}}

Looking at the info above, I have decided to reduce the feature space to just 'Day', which I think will be sufficient for this purpose.
Actually, I will also include the Year values into a column called PrecipitationYTD. This should give offer the model insight into both
granular changes over the days and also long term trends of precipitation.

We will:
- extract the Day precipitation (inches) into a list
- extract the Year precipitation(inches) into a list
- insert both lists as df's and remove the existing "Precipitation" column

In [59]:
day_precips, year_precips = [], []

for i in range(len(weather_data)):
    precip_data = eval(weather_data.iloc[i]["Precipitation"])
    day_precips.append(precip_data['Day']["In"])
    year_precips.append(precip_data["Year"]["In"])

### check that precips data and weather data match up lengthwise
print("all df lengths equal")
len(day_precips) == len(year_precips) == len(weather_data)

precip_col_index = list(weather_data.columns).index("Precipitation")

weather_data.insert(precip_col_index, "PrecipYr(in)", pd.DataFrame(year_precips))
weather_data.insert(precip_col_index, "PrecipDay(in)", pd.DataFrame(day_precips))
weather_data.drop(["Precipitation"], axis=1, inplace=True)

weather_data.sample(3)

all df lengths equal


,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure,PrecipDay(in),PrecipYr(in),WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
4727,1627512900,7/28/2021 17:55,282255Z AUTO 25003KT 10SM OVC029 31/25 A2998 R...,467966856,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 88, 'Celsius': 31}","{'Fahrenheit': 77, 'Celsius': 25}","{'Type': 'HeatIndex', 'Fahrenheit': 100, 'Cels...","{'Fahrenheit': 100, 'Celsius': 38}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1015.24, 'InchesOfMercury': 29.98}",0.0,3.49,3.4,West,0.704,10.0,Overcast Sky,05,20
6148,1657859700,7/14/2022 23:35,150435Z 06003KT 10SM CLR 21/17 A3016 RMK AO2 T...,524315191,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 70, 'Celsius': 21}","{'Fahrenheit': 63, 'Celsius': 17}","{'Type': None, 'Fahrenheit': 70, 'Celsius': 21}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1021.34, 'InchesOfMercury': 30.16}",0.0,7.10,3.4,East,0.780,10.0,Clear Skies,05,20
2053,1569020100,9/20/2019 17:55,202255Z AUTO 00000KT 10SM CLR 29/22 A3009 RMK ...,325515098,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 84, 'Celsius': 29}","{'Fahrenheit': 72, 'Celsius': 22}","{'Type': 'HeatIndex', 'Fahrenheit': 89, 'Celsi...","{'Fahrenheit': 89, 'Celsius': 32}","{'Fahrenheit': None, 'Celsius': None}","{'Millibar': 1018.97, 'InchesOfMercury': 30.09}",0.0,14.55,0.0,North,0.660,10.0,Clear Skies,06,18


Moving on. Let's move on to

#### Column: Pressure

This one should be pretty easy. Let's take the 'Millibar' measurement, as it seems more consistently formatted

In [60]:
### initialize empty millibars break

millibars = []

for i in range(len(weather_data)):
    pressure_data = eval(weather_data.iloc[i]["Pressure"])
    millibars.append(pressure_data['Millibar'])

weather_data['Pressure'] = pd.DataFrame(millibars)
weather_data.rename(columns={'Pressure' : 'Pressure(mb)'}, inplace=True)
weather_data.sample()

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike,HeatIndex,Windchill,Pressure(mb),PrecipDay(in),PrecipYr(in),WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
2766,1585608900,3/30/2020 17:55,302255Z AUTO 02007KT 10SM OVC045 11/02 A3012 R...,366602183,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 52, 'Celsius': 11}","{'Fahrenheit': 36, 'Celsius': 2}","{'Type': None, 'Fahrenheit': 52, 'Celsius': 11}","{'Fahrenheit': None, 'Celsius': None}","{'Fahrenheit': None, 'Celsius': None}",1019.98,0.02,1.21,8.1,North,0.541,10.0,Overcast Sky,06,19


Moving on. Let's move on to

#### Column: Windchill

In [61]:
windchill_data = weather_data.iloc[:,9]
windchill_data.value_counts()

Windchill
{'Fahrenheit': None, 'Celsius': None}    5137
{'Fahrenheit': 29, 'Celsius': -2}         100
{'Fahrenheit': 42, 'Celsius': 6}           98
{'Fahrenheit': 25, 'Celsius': -4}          95
{'Fahrenheit': 33, 'Celsius': 1}           88
{'Fahrenheit': 27, 'Celsius': -3}          86
{'Fahrenheit': 35, 'Celsius': 2}           82
{'Fahrenheit': 30, 'Celsius': -1}          82
{'Fahrenheit': 22, 'Celsius': -6}          82
{'Fahrenheit': 44, 'Celsius': 7}           80
{'Fahrenheit': 36, 'Celsius': 2}           75
{'Fahrenheit': 40, 'Celsius': 4}           73
{'Fahrenheit': 31, 'Celsius': -1}          65
{'Fahrenheit': 41, 'Celsius': 5}           64
{'Fahrenheit': 45, 'Celsius': 7}           63
{'Fahrenheit': 24, 'Celsius': -4}          63
{'Fahrenheit': 39, 'Celsius': 4}           62
{'Fahrenheit': 38, 'Celsius': 3}           62
{'Fahrenheit': 21, 'Celsius': -6}          60
{'Fahrenheit': 37, 'Celsius': 3}           56
{'Fahrenheit': 26, 'Celsius': -3}          50
{'Fahrenheit': 34, 'Cels

I think the Windchill and Heatindex columns might be related somehow. Let's look at the value counts for HeatIndex and see if we can't discover something

In [62]:
heatindex_data = weather_data.iloc[:, 8]
heatindex_data.value_counts()

HeatIndex
{'Fahrenheit': None, 'Celsius': None}    6641
{'Fahrenheit': 84, 'Celsius': 29}          66
{'Fahrenheit': 82, 'Celsius': 28}          57
{'Fahrenheit': 85, 'Celsius': 29}          56
{'Fahrenheit': 86, 'Celsius': 30}          55
{'Fahrenheit': 88, 'Celsius': 31}          53
{'Fahrenheit': 83, 'Celsius': 28}          45
{'Fahrenheit': 87, 'Celsius': 31}          45
{'Fahrenheit': 81, 'Celsius': 27}          39
{'Fahrenheit': 93, 'Celsius': 34}          31
{'Fahrenheit': 89, 'Celsius': 32}          29
{'Fahrenheit': 92, 'Celsius': 33}          27
{'Fahrenheit': 96, 'Celsius': 36}          23
{'Fahrenheit': 91, 'Celsius': 33}          23
{'Fahrenheit': 95, 'Celsius': 35}          20
{'Fahrenheit': 97, 'Celsius': 36}          19
{'Fahrenheit': 90, 'Celsius': 32}          17
{'Fahrenheit': 98, 'Celsius': 37}          13
{'Fahrenheit': 94, 'Celsius': 34}          12
{'Fahrenheit': 104, 'Celsius': 40}          9
{'Fahrenheit': 100, 'Celsius': 38}          9
{'Fahrenheit': 101, 'Cel

Let's change our plans. Upon further examination, it's not "Windchill" and "HeatIndex" that are related,  
but both of these columns are represented in our "FeelsLike" column. Since the temp value in the "FeelsLike" 
column will imply Windchill or Heatindex (lower or higher than recorded temp), We're going to drop the Windchill and HeatIndex and Columns altogether.
  
This will neatly take care of the thousands of missing values in both "Windchill" and "HeatIndex" as they will be represented by the "FeelsLike" temp  
being the same as the main recorded temp



In [63]:
### Let's first change the "FeelsLike" column to reflect the Farenheight temperature

feels_like = []

for i in range(len(weather_data)):
    feels_like_data = eval(weather_data.iloc[i]["FeelsLike"])
    feels_like.append(feels_like_data["Fahrenheit"])

# check that lengths match up

print(len(feels_like) == len(weather_data))

weather_data["FeelsLike"] = pd.DataFrame(feels_like)
weather_data.rename(columns={"FeelsLike" : "FeelsLike(F)"}, inplace=True)
weather_data.drop(["HeatIndex", "Windchill"], axis=1, inplace=True)
weather_data.head(2)

True


,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature,Dewpoint,FeelsLike(F),Pressure(mb),PrecipDay(in),PrecipYr(in),WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
0,1525172100,5/1/2018 5:55,011055Z AUTO 19004KT 10SM CLR 13/03 A3000 RMK ...,220160752,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 55, 'Celsius': 13}","{'Fahrenheit': 37, 'Celsius': 3}",55,1015.92,0.0,4.44,4.6,South,0.509,10.0,Clear Skies,05,19
1,1525193700,5/1/2018 11:55,011655Z AUTO 23013G23KT 10SM SCT065 27/11 A299...,220215015,"{'timezone': 'America/Chicago', 'string_local'...","{'Fahrenheit': 81, 'Celsius': 27}","{'Fahrenheit': 52, 'Celsius': 11}",81,1015.58,0.0,4.44,15.0,South,0.368,10.0,Scattered,05,19


Only a few more important columns to go! Let's move on to 

#### Columns: Dewpoint and Temperature

We will extract the Fahrenheit temp from both columns, easy enough

In [64]:
temperatures, dewpoints = [], []

for i in range(len(weather_data)):
    temp_data = eval(weather_data.iloc[i]["Temperature"])
    dew_data = eval(weather_data.iloc[i]["Dewpoint"])

    temperatures.append(float(temp_data["Fahrenheit"]))
    dewpoints.append(dew_data["Fahrenheit"])

weather_data["Temperature"] = pd.DataFrame(temperatures)
weather_data["Dewpoint"] = pd.DataFrame(dewpoints)

weather_data.rename(columns = {"Temperature" : "Temperature(F)", "Dewpoint" : "Dewpoint(F)"}, inplace=True)

The last thing we need to do is configure the datetime column. Let's see what it has inside

#### Column: datetime

In [65]:
weather_data.iloc[i]["datetime"]

"{'timezone': 'America/Chicago', 'string_local': '20230504231500', 'iso_8601_local': '2023-05-04T23:15:00-05:00', 'is_local_night': True, 'string_utc': '20230505041500', 'iso_8601_utc': '2023-05-05T04:15:00+00:00', 'unix_timestamp': 1683260100}"

Let's use the 'iso_8601_local' values for our datetime column

In [66]:
datetimes = []

for i in range(len(weather_data)):
    datetime_data = eval(weather_data.iloc[i]["datetime"])
    iso_8601_local = datetime_data["iso_8601_local"]
    
    datetimes.append(pd.to_datetime(iso_8601_local))

weather_data["datetime"] = pd.DataFrame(datetimes)
weather_data["datetime"][0]

Timestamp('2018-05-01 05:55:00-0500', tz='UTC-05:00')

In [67]:
weather_data.sample()

,ObservedAt,ObservedAt_DateTime,Record,ID,datetime,Temperature(F),Dewpoint(F),FeelsLike(F),Pressure(mb),PrecipDay(in),PrecipYr(in),WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
2360,1575870900,12/8/2019 23:55,090555Z AUTO 17003KT 10SM SCT016 BKN037 OVC120...,342366667,2019-12-08 23:55:00-06:00,48.0,48,48,1006.1,0.0,15.44,3.4,South,1.0,10.0,Scattered,07,16


Ok. Let's rename the datetime column to "ObservationDateTime" and drop the columns "ID", "Record", "ObservedAt_DateTime", and "ObservedAt."

In [68]:
weather_data.rename({"datetime" : "ObservationDateTime"}, axis=1, inplace=True)
weather_data.drop(["ID","Record","ObservedAt_DateTime","ObservedAt"], axis=1, inplace=True)

### We're done cleaning and parsing the data!

Let's take another overview of our more useful dataset, and compare to the original, raw dataset

In [69]:
weather_data.sample(5)

,ObservationDateTime,Temperature(F),Dewpoint(F),FeelsLike(F),Pressure(mb),PrecipDay(in),PrecipYr(in),WindSpeed(mph),WindDirection,RelativeHumidity,Visibility(mi),Conditions,SunRise(HH),SunSet(HH)
3227,2020-07-21 23:55:00-05:00,73.0,72,73,1014.23,0.00,5.58,0.0,North,0.941,10.0,Clear Skies,05,20
3989,2021-01-26 05:55:00-06:00,32.0,30,24,1006.10,0.00,0.25,9.2,North,0.931,5.0,Overcast Sky,07,17
4034,2021-02-06 11:55:00-06:00,14.0,1,3,1020.66,0.00,0.25,8.1,West,0.569,10.0,Scattered,06,17
2717,2020-03-18 11:55:00-05:00,45.0,41,42,1019.65,0.02,1.21,4.6,East,0.872,5.0,Broken Sky,06,19
6026,2022-06-14 10:35:00-05:00,90.0,77,104,1011.52,0.00,5.52,11.5,South,0.665,10.0,Clear Skies,05,20


In [70]:
print(weather_data_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7324 entries, 0 to 7323
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ObservedAt                     7324 non-null   int64  
 1   ObservedAt_DateTime            7324 non-null   object 
 2   Station                        7324 non-null   object 
 3   Record                         7324 non-null   object 
 4   ID                             7324 non-null   int64  
 5   datetime                       7324 non-null   object 
 6   Temperature                    7324 non-null   object 
 7   Dewpoint                       7324 non-null   object 
 8   FeelsLike                      7324 non-null   object 
 9   HeatIndex                      7324 non-null   object 
 10  Windchill                      7324 non-null   object 
 11  Pressure                       7324 non-null   object 
 12  Precipitation                  7324 non-null   o

In [71]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7324 entries, 0 to 7323
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObservationDateTime  7324 non-null   object 
 1   Temperature(F)       7324 non-null   float64
 2   Dewpoint(F)          7324 non-null   int64  
 3   FeelsLike(F)         7324 non-null   int64  
 4   Pressure(mb)         7309 non-null   float64
 5   PrecipDay(in)        7324 non-null   float64
 6   PrecipYr(in)         7324 non-null   float64
 7   WindSpeed(mph)       7324 non-null   float64
 8   WindDirection        7324 non-null   object 
 9   RelativeHumidity     7324 non-null   float64
 10  Visibility(mi)       7324 non-null   float64
 11  Conditions           7324 non-null   object 
 12  SunRise(HH)          7324 non-null   object 
 13  SunSet(HH)           7324 non-null   object 
dtypes: float64(7), int64(2), object(5)
memory usage: 801.2+ KB


# Final Notes

We have a cleaned, parsed dataset that is roughly half the size of the original, raw dataset. To sum up, we:  
- parsed data structures within the column data to extract relevant features
- handled missing columnar data in the following ways:  
//// Imputed the missing data using the mode of the column data ("WindDirection", "Conditions")  
//// Referenced related column values at the same index to impute NaN values ("WindDirection", "Conditions")
- created new / renamed columns for new features, dropped irrelevant, redundant, and empty columns  
  
### Export the cleaned dataset to csv for use by the visualization and training notebooks

In [72]:
weather_data.to_csv("cleaned_weather_data_1.csv", index=False)